In [183]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.stats import norm
import scipy.stats
from scipy.stats import ncx2

new_line = '\n'
pd.options.mode.chained_assignment = None 
import warnings
warnings.filterwarnings('ignore')

sns.set(font_scale=1.5, rc={'text.usetex' : True,})
%config InlineBackend.figure_format='retina'

## 1 Vasicek model
$ dr_t =\kappa ( \bar{r} - r)dt+\sigma dW$

In [184]:
r0 = 0.05
sigma = 0.1
kappa = 0.82
r_bar = 0.05

In [213]:
def Vasicek_R(T,t,path,r0,kappa,r_bar,sigma):
    
#     np.random.seed(100)
    
    steps = int((T-t)*360)
    dt = (T-t) / steps
    rt = np.zeros((path,steps+1))
    rt[:,0] = r0
    for i in range(1,steps+1):
        dWt = np.random.normal(0,1,path)
        rt[:,i] = rt[:,i-1] + kappa*(r_bar - rt[:,i-1])*dt + sigma * np.sqrt(dt) * dWt
        
    return rt

In [214]:
def ZeroCouponBond(T,t,path,r0,kappa,r_bar,sigma,FaceValue):
    
    steps = int((T-t)*360)
    dt = (T-t) / steps
    rt = Vasicek_R(T,t,path,r0,kappa,r_bar,sigma)
    
    disc = np.exp( - np.sum(rt,axis=1) * dt)
    price = np.mean(disc * FaceValue ) 
    
    return price

### (a)  Monte Carlo Simulation for pure discount bond, with Face Value of $1,000, maturing in 𝑇 = 0.5 years

In [215]:
T = 0.5
t=0
FV=1000

price_a = ZeroCouponBond(T=T,t=t,path=1000,r0=r0,kappa=kappa,r_bar=r_bar,sigma=sigma,FaceValue=FV)

In [216]:
print(f"Monte Carlo Simulation for pure discount bond is {round(price_a,4)}")

Monte Carlo Simulation for pure discount bond is 975.2247


### (b) Monte Carlo Simulation to find the price of a coupon-paying bond, with Face Value of 1,000, paying semiannual coupons of 30, maturing in T = 4 years

In [217]:
T=4
coupon = np.append(np.repeat(30,7),1030)
time = np.arange(0.5,4.5,0.5)

def CPB(T,t,path,r0,kappa,r_bar,sigma,FaceValue):
    
    price = 0
    for i in range(len(coupon)):
        
        p= ZeroCouponBond(T=t[i],t=0,path=path,r0=r0,kappa=kappa,r_bar=r_bar,sigma=sigma,FaceValue=FaceValue[i])
        price += p
   
    return price

In [218]:
price_b = CPB(T=T,t=time,path=1000,r0=r0,kappa=kappa,r_bar=r_bar,sigma=sigma,FaceValue=coupon)

In [219]:
print(f"Monte Carlo Simulation for a coupon-paying bond is {round(price_b,4)}")

Monte Carlo Simulation for a coupon-paying bond is 1039.3391


### (c) Use Monte Carlo Simulation to find the price of a European Call Option on the Pure Discount Bond of part (a). The option expires in 3 months and has a strike price of K = 980. Use the explicit formula for the underlying bond price (use explicit formula only for the bond price).


In [220]:
def BondPrice_Explicit(kappa,T,t,r_bar,sigma,rt):
    """
    Bond price using explicit formula
    """
    
    B = 1/kappa * (1- np.exp(-kappa*(T-t)))
    A = np.exp((r_bar - sigma**2/(2*kappa**2))*(B-(T-t)) - sigma**2/(4*kappa)*B**2)
    
    return A*np.exp(-B*rt)

In [236]:
def EuroCall_ZCP(kappa,T,t,r_bar,sigma,path,r0,FaceValue,K):
    
    steps = int((T-t)*360)
    dt = (T-t) / steps
    rt = Vasicek_R(T=T,t=t,path=path,r0=r0,kappa=kappa,r_bar=r_bar,sigma=sigma)
    rt = rt[:,-1]
    
    bondp = BondPrice_Explicit(kappa=kappa,T=T,t=t,r_bar=r_bar,sigma=sigma,rt=rt)
    bondp = bondp * FaceValue
    
    payoff = np.maximum(bondp - K,0)
    price = np.mean(payoff)
    
    return price

In [237]:
K = 980
T = 0.5
t = 3/12
price_c = EuroCall_ZCP(T=T,t=t,path=1000,r0=r0,kappa=kappa,r_bar=r_bar,sigma=sigma,FaceValue=FV,K=K)

In [238]:
print(f"Monte Carlo Simulation of European call for a zero coupon bond with explicit method is {round(price_c,4)}")

Monte Carlo Simulation of European call for a zero coupon bond with explicit method is 9.1768


### (d) Use Monte Carlo Simulation to find the price of a European Call Option on the coupon-paying bond in part (b). The option expires in 3 months and has a strike price of 𝐾 = 980. Use Monte Carlo simulation for pricing the underlying bond (no explicit formula can be used in this part).

In [224]:
T=4
coupon = np.append(np.repeat(30,7),1030)
time = np.arange(0.5,4.5,0.5)

def EuroCall_CPB(T,t,path,r0,kappa,r_bar,sigma,FaceValue,K):
    
    price = 0
    for i in range(len(coupon)):
        
        p= ZeroCouponBond(T=t[i],t=0,path=path,r0=r0,kappa=kappa,r_bar=r_bar,sigma=sigma,FaceValue=FaceValue[i])
        price += p
        
    price = np.maximum(price-K,0)
   
    return price


In [232]:
K=980

EuroCall_CPB(T=T,t=time,path=1000,r0=r0,kappa=kappa,r_bar=r_bar,sigma=sigma,FaceValue=coupon,K=980)

60.63548007430518

### (e) Find the price of a European Call option of part (d) by using the explicit formula for the underlying bond price, and reconcile the findings here with the ones of part (d).

## 2 CIR Model

$ dr =\kappa(\bar{r}-r)dt+\sigma\sqrt{r}dW $

### (a) Use Monte Carlo Simulation to find at time t = 0 the price c(t, T, S) of a European Call option, with strike price of K = 980 and expiration in T = 0.5 years on a Pure Discount Bond with Face Value of 1,000, that matures in S = 1 year:

### (b)  Use the Implicit Finite-Difference Method to find at time t = 0 the price c(t, T, S) of a European Call option, with strike price of S = 980 and expiration in T = 0.5 years on a Pure Discount Bond with Face Value of 1,000, which matures in S = 1 year. 

### (c) Compute the price c(t, T, S) of the European Call option above using the explicit formula, and compare it to your findings in parts (a) and (b) and comment on your findings.

## 3. G2++ model